# Load our church main src

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Clear the church member data source content while keeping the columns names only

In [ ]:
# import csv

# # Specify the CSV file path
# csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Members_Attendee_Src'

# # Read the column names from the existing CSV file
# with open(csv_file_path, 'r') as file:
#     reader = csv.reader(file)
#     column_names = next(reader)  # Read the first row as column names

# # Write the column names back to the CSV file
# with open(csv_file_path, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(column_names)

# print("CSV file contents cleared while keeping column names.")


# Connecting a Google Colab notebook with the Anvil web app

---

### This notebook is designed to be used alongside Anvil's [turning a Colab notebook into a web app tutorial](https://anvil.works/learn/tutorials/google-colab-to-web-app).

The text cells below tell you the steps you need to take to connect this notebook to an Anvil app. The steps are:


1. Install the `anvil-uplink` library
2. Import the `anvil.server` package
3. Connect the notebook using your apps Uplink key
4. Create a function to call from your app that includes the `anvil.server.callable` decorator
5. Add `anvil.server.wait_forever()` to the end of the notebook

### Follow along below for more detail.

---





### Let's start by installing the `anvil-uplink` library, all we need to do is add `!pip install anvil-uplink` to the first cell of the notebook:

In [ ]:
!pip install anvil-uplink

### Next import the Anvil server package by adding `import anvil.server`:

Importing `anvil.server` means, when this notebook is connected via the Uplink, it will behave in the same way as any other [Anvil Server Module](https://anvil.works/docs/server).

In [ ]:
import anvil.server

### Then connect this notebook to your app using your Uplink key `anvil.server.connect("your-uplink-key")`:

For information on how to get your apps Uplink key, see [Step 4 - Enable the Uplink](https://anvil.works/learn/tutorials/google-colab-to-web-app#step-4-enable-the-uplink).

In [ ]:
anvil.server.connect("server_WNFXJZNRL4C3V527M2KYJZCT-IAKEVNEL4STXNXOB")

### Install the pandas package


In [ ]:
import pandas as pd


### Create a data frame that stores or append the new / old church members (without taking null church ID number / visitors data)

In [ ]:
import datetime
import os

members_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Members_src')

today = datetime.date.today()
Current_Date = today.strftime("%d %B %Y")


csv_file_path = "/content/drive/MyDrive/Colab Notebooks/Members_Attendee_Src"


header_value = not os.path.exists("/content/drive/MyDrive/Colab Notebooks/Members_src")  # Set header if file doesn't exist

# Read the existing CSV file (if it exists)
members_attendee__df = pd.read_csv(csv_file_path) if os.path.exists(csv_file_path) else None

# Dealing more with data frame for the church/visitor members, filter or negate rows in members source or with visitor status 'Not Yet'
members_df = members_df[~members_df['Church Reference'].isin(members_attendee__df['Church Reference'])]

members_df = members_df[~(members_df['Join Church'] == 'Not Yet')]

members_df.to_csv(csv_file_path, mode='a', header=header_value, index=False)


# Optionally, update the existing_df to include the appended rows
if  members_attendee__df is not None:
    members_df = members_attendee__df.append(members_df, ignore_index=True)
else:
    members_df = members_attendee__df

members_df[Current_Date] = pd.NA

members_df.head()

###Create a function that generates the user details linked to the church members ID

In [ ]:
from pandas.io.pytables import IndexCol
from numpy.lib import index_tricks


df_church_members = members_df


def confirm_church_ref_(church_ref):

  global attendee_members_df
  attendee_matches_df = df_church_members[df_church_members['Church Reference'] == str(church_ref)].copy()

  print(attendee_matches_df.columns)

  if attendee_matches_df.empty:

    return "InvalidRef"
    # return "0"
  else:
    title = 'Mr.'
    if attendee_matches_df['Gender'].iloc[0] == 'Female':
      title = 'Miss/Mrs.'


    # members_attendee_df = pd.concat([members_attendee_df,attendee_matches_df], ignore_index= True)
    capitilized_name = title+" "+attendee_matches_df['First Name'].str.capitalize() +" "+attendee_matches_df['Last Name'].str.capitalize()
    print(capitilized_name)
    return (capitilized_name.to_string(index = False, header = False))




### Determine if user Exist, logged-in to system today, or does not exist

In [ ]:
def user_exist_search(ext_num):
  import datetime

  # Note:
  # ext_num / Church Reference refers to id_num / Church ID Number



  today = datetime.date.today()
  Current_Date = today.strftime("%d %B %Y")


  matching_rows = members_df.loc[members_df['Church Reference'] == ext_num]
  matching_rows_yes_member = members_df.loc[(members_df['Church Reference'] == ext_num) & (members_df[Current_Date] == 'Yes')]
  # matching_rows_curr_date = members_attendee_df.loc[(members_attendee_df['Extension Number'] == ext_num) & (members_attendee_df['Current Date'] == current_date)]

  if matching_rows.empty:
    print('matching_rows.empty()')
    return('no user exist')


  else:
    matching_row_df = matching_rows.iloc[0]
    # matching_rows_yes_memeber =
    matching_row_df_ = pd.DataFrame([matching_row_df])
    matching_row_df_ = matching_row_df_.reset_index(drop=True)

    try:
      if  matching_rows.empty == False and  matching_row_df[Current_Date] == 'Yes':
        print('not matching_rows.empty() and  not matching_row')
        return('User exist')

    except Exception as e:
      print('exception '+str(e))

    try:
      # if matching_rows[(([matching_rows['current_date'] == 'Not Yet']) and ([matching_rows['Current Date'] == current_date]))].empty and join_church == 'Yes':
      if matching_rows_yes_member.empty == True and matching_rows.empty == False:
        index_value = matching_rows.index[0]
        print('update_member_status(index_value, current_date)')
        update_member_status(index_value, Current_Date)

        return('True')

    except Exception as e:
      print('exception2 '+str(e))


### Mark user as attended based on the current date

In [ ]:
def update_member_status(index_val, current_date):
  members_df.at[index_val, current_date] = 'Yes'
  print(f"members_df: {members_df}")




### Get and Print the church ref number

In [ ]:
# Create frunction to add members or visitors and with a terminal value to get end results
@anvil.server.callable
def confirm_church_ref(church_ref):
  user_exist_status = user_exist_search(church_ref)

  if user_exist_status ==  'True':
    confirm_names = confirm_church_ref_(str(church_ref))
    return (confirm_names)
  else:
    if user_exist_status == 'User exist':
      return('User exist')
    else:
      return('no user exist')





confirm_church_ref('TD0000183')

#  print(str(sit_number))





### test

In [ ]:
members_df.head()

### test(Call function called by the anvil client server here)

In [ ]:
# confirm_church_ref('TD0000183')

### Finally, let's add `anvil.server.wait_forever()` function so the notebook is always available to the web app:

In [ ]:
import anvil.server
import time

# Define the duration in seconds after which the server should stop
duration = 1200  # 6 seconds

def stop_server():
    print("Stopping the server...")
    anvil.server.disconnect()

# Sleep for the specified duration before stopping the server
time.sleep(duration)
stop_server()
try:
  anvil.server.wait_forever()
except:
  pass



###End use interface

##Save the data on cloud in CSV format with a current date name convention and generate description and information

In [ ]:
from datetime import datetime
import os

current_datetime = datetime.now()
# print(current_datetime)
formatted_datetime = current_datetime.strftime("%A, %B, %d, %Y")


csv_file_path = "/content/drive/MyDrive/Colab Notebooks/Members_Attendee_Src"

print(csv_file_path)
header_value = False
try:
  df_temp = pd.read_csv(csv_file_path)

except pd.errors.EmptyDataError:
  header_value = True

members_df = members_df.fillna('')

members_df = members_df.dropna(how='all')

members_df.to_csv(csv_file_path, header=True, index=False)
members_df.head()


### Summery of church attendance at church

In [ ]:
import numpy as np
from google.colab import autoviz
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
    df[colname].value_counts().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
    plt.gca().spines[['top', 'right',]].set_visible(False)
    return autoviz.MplChart.from_current_mpl_state()

# List of consecutive columns you want to visualize
last_date_col = members_df.shape[1]
consecutive_columns = members_df.columns[9:last_date_col]  # Adjust this range according to your data

# Set a larger figure size
figsize = (2, 1)

# Loop through the consecutive columns and create categorical histograms
for col in consecutive_columns:
    plt.figure(figsize=figsize)  # Set figure size
    # print(col)
    # colname = members_df.columns[int(col)]
    # df[colname].value_counts().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
    plt.gca().spines[['top', 'right',]].set_visible(False)
    plt.title(f'Members Attendees for {col} church service')
    # plt.xlabel('Members count')
    # plt.ylabel("Attend Response")
    plt.tight_layout()
    chart = categorical_histogram(members_df, col)
    chart.display()


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import numpy as np
from google.colab import autoviz
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
# from google.colab import files

# Define the members_df and other variables as needed

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
    df[colname].value_counts().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
    plt.gca().spines[['top', 'right']].set_visible(False)
    return plt  # Return the plt object for further customization if needed

# List of consecutive columns you want to visualize
last_date_col = members_df.shape[1]
consecutive_columns = members_df.columns[9:last_date_col]  # Adjust this range according to your data

# Set a larger figure size
figsize = (2, 1)

# Create a folder to save the charts
!mkdir charts

# Loop through the consecutive columns and create categorical histograms
for col in consecutive_columns:
    plt.figure(figsize=figsize)  # Set figure size
    plt.gca().spines[['top', 'right']].set_visible(False)
    plt.title(f'Members Attendees for {col} church service')
    plt.tight_layout()
    chart = categorical_histogram(members_df, col)

    # Save the chart as an image file
    chart_path = f"charts/chart_{col}.png"
    plt.savefig(chart_path)
    plt.close()  # Close the plot

    # chart.display()  # Display the chart

# Move all generated charts to a subdirectory
!mv *.png charts/

# Optional: Zip the charts folder
!zip -r charts.zip charts/


### Each day Data on a seperated graphical pivotal chart display

In [ ]:
# Keep / is per each day on seperate
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import zipfile
from google.colab import files

def send_email(subject, body, attachment_path):
    # Set up the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_username = "dummy2006dummy2006@gmail.com"  # Replace with your Gmail email
    smtp_password = "azmlpzgdmlcprzee"  # Replace with your Gmail app password

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    # Create the email content
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = smtp_username  # Sending the email to yourself
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Attach the ZIP file
    zip_attachment_path = "charts.zip"
    with open(zip_attachment_path, "rb") as f:
        zip_attachment = MIMEBase("application", "zip")
        zip_attachment.set_payload(f.read())
        encoders.encode_base64(zip_attachment)
        zip_attachment.add_header(
            "Content-Disposition", f"attachment; filename={zip_attachment_path}"
        )
        msg.attach(zip_attachment)


    # Send the email
    server.sendmail(smtp_username, smtp_username, msg.as_string())
    server.quit()



# Rest of your code...

attachments = []
figsize = (2, 1)
for col in consecutive_columns:
    plt.figure(figsize=figsize)
    plt.gca().spines[["top", "right"]].set_visible(False)
    plt.title(f"Members Attendees for {col} church service")
    # plt.tight_layout()
    chart = categorical_histogram(members_df, col)

    # Save the chart as an image
    attachment_path = f"chart_{col}.jpeg"
    attachments.append(attachment_path)
    plt.savefig(attachment_path)
    plt.close()  # Close the plot

# Create a ZIP file containing all attachments
with zipfile.ZipFile("charts.zip", "w") as zipf:
    for attachment in attachments:
        zipf.write(attachment)

# Send the email with the ZIP attachment
send_email("Churcha Attendance Report", "Good day\n\nPlease find the attached church attendance report.\n\nKR\nADMIN", "charts.zip")


### Send an email

In [ ]:
### All results of all days grouped together
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import zipfile
from google.colab import files

def send_email(subject, body, attachment_path):
    # Set up the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_username = "dummy2006dummy2006@gmail.com"  # Replace with your Gmail email
    smtp_password = "azmlpzgdmlcprzee"  # Replace with your Gmail app password

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    # Create the email content
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = smtp_username  # Sending the email to yourself
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    zip_attachment_path = "charts.zip"
    with open(zip_attachment_path, "rb") as f:
        zip_attachment = MIMEBase("application", "zip")
        zip_attachment.set_payload(f.read())
        encoders.encode_base64(zip_attachment)
        zip_attachment.add_header(
            "Content-Disposition", f"attachment; filename={zip_attachment_path}"
        )
        msg.attach(zip_attachment)



    # Send the email
    server.sendmail(smtp_username, smtp_username, msg.as_string())
    server.quit()


# Rest of your code...

# Create a single clustered plot with all columns
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.title("Members Attendees for Church Services")
plt.gca().spines[["top", "right"]].set_visible(False)
sns.set_palette("Dark2")  # Set palette for colors
for col in consecutive_columns:
    categorical_histogram(members_df, col, figscale=0.8)

plt.legend(consecutive_columns)
plt.tight_layout()

# Save the clustered plot as an image
clustered_plot_path = "clustered_plot.png"
plt.savefig(clustered_plot_path)
plt.close()  # Close the plot

# Create a ZIP file containing the clustered plot
with zipfile.ZipFile("charts.zip", "w") as zipf:
    zipf.write(clustered_plot_path, "clustered_plot.png")

# Send the email with the ZIP attachment
send_email("Church Attendance Reports", "Good day\n\nPlease find the attached church attendance reports.\n\nKR\nADMIN", "charts.zip")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create subplots for each column
fig, axes = plt.subplots(nrows=len(consecutive_columns), ncols=1, figsize=(10, 8))

# Loop through consecutive_columns and create countplots
for i, col in enumerate(consecutive_columns):
    sns.countplot(data=members_df, x=col, ax=axes[i])
    axes[i].set_title(f"{col} Church Service")
    axes[i].set_ylabel("Count")

plt.tight_layout()
plt.show()


In [ ]:
# TO SEND or do THE ABOVE
#  import numpy as np
# from matplotlib import pyplot as plt
# import seaborn as sns
# from IPython.display import display
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from email.mime.base import MIMEBase
# from email import encoders
# import zipfile

# # Function to send the email
# def send_email(subject, body, attachment_path):


# # Set up the email details
#     smtp_server = "smtp.gmail.com"
#     smtp_port = 587
#     smtp_username = "dummy2006dummy2006@gmail.com"  # Replace with your Gmail email
#     smtp_password = "azmlpzgdmlcprzee"  # Replace with your Gmail app password

# # Create the email content
#     msg = MIMEMultipart()
#     msg["From"] = smtp_username
#     msg["To"] = smtp_username  # Sending the email to yourself
#     msg["Subject"] = "Visualization Attachments"    # ... (your email setup code) ...

#     # Attach the ZIP file
#     zip_attachment_path = "charts.zip"
#     with open(zip_attachment_path, "rb") as f:
#         zip_attachment = MIMEBase("application", "zip")
#         zip_attachment.set_payload(f.read())
#         encoders.encode_base64(zip_attachment)
#         zip_attachment.add_header(
#             "Content-Disposition", f"attachment; filename={zip_attachment_path}"
#         )
#         msg.attach(zip_attachment)

#     # Send the email
#     server.sendmail(smtp_username, smtp_username, msg.as_string())
#     server.quit()

# # Rest of your code...

# # Create a single clustered plot with all columns
# plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
# plt.title("Members Attendees for Church Services")
# plt.gca().spines[["top", "right"]].set_visible(False)
# sns.set_palette("Dark2")  # Set palette for colors

# # Loop through consecutive_columns and create clustered plots
# for col in consecutive_columns:
#     plt.subplot(len(consecutive_columns), 1, consecutive_columns.get_loc(col) + 1)
#     categorical_histogram(members_df, col, figscale=0.8)
#     plt.title(f"{col} Church Service")
#     plt.xlabel("")  # Clear x-axis label for better alignment

# plt.tight_layout()
# plt.show()

# # Save the clustered plot as an image
# clustered_plot_path = "clustered_plot.png"
# plt.savefig(clustered_plot_path)
# plt.close()  # Close the plot

# # Create a ZIP file containing the clustered plot
# with zipfile.ZipFile("charts.zip", "w") as zipf:
#     zipf.write(clustered_plot_path, "clustered_plot.png")

# # Send the email with the ZIP attachment
# send_email("Generated Charts", "Here are the generated charts.", "charts.zip")


---

## That's it, 5 simple steps to connect your notebook to your Anvil app!

---

### Run the notebook. You should see output like this:

Connecting to wss://anvil.works/uplink

Anvil websocket open

Authenticated OK


# Test(Take back the data source to drive)

In [ ]:
# drive.flush_and_unmount()